In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import urllib.request
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
import plotnine as p9

In [2]:
data = pd.read_csv(os.path.expanduser('../Data/COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv')).head(30000)
data.head()

/var/folders/0x/j2sfsgp167vf2w83dyjt6_nm0000gn/T/ipykernel_13560/436026595.py:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.


,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum
0,052052,2020/05/29,CA,052052,KINDRED HOSPITAL RIVERSIDE,2224 MEDICAL CENTER DRIVE,PERRIS,92571.0,Long Term,6065.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
1,070038,2020/04/24,CT,070038,"CONNECTICUT HOSPICE INC,THE",100 DOUBLE BEACH ROAD,BRANFORD,6405.0,Short Term,9009.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
2,100034,2020/05/01,FL,100034,"MOUNT SINAI MEDICAL CENTER OF FLORIDA, INC",4300 ALTON RD,MIAMI BEACH,33140.0,Short Term,12086.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
3,100069,2020/05/15,FL,100069,ADVENTHEALTH CARROLLWOOD,7171 N DALE MABRY HWY,TAMPA,33614.0,Short Term,12057.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
4,100070,2020/04/24,FL,100070,SHOREPOINT HEALTH VENICE,540 THE RIALTO,VENICE,34285.0,Short Term,12115.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN


In [3]:
len(data)

30000

In [4]:
data.columns.to_list()

['hospital_pk',
 'collection_week',
 'state',
 'ccn',
 'hospital_name',
 'address',
 'city',
 'zip',
 'hospital_subtype',
 'fips_code',
 'is_metro_micro',
 'total_beds_7_day_avg',
 'all_adult_hospital_beds_7_day_avg',
 'all_adult_hospital_inpatient_beds_7_day_avg',
 'inpatient_beds_used_7_day_avg',
 'all_adult_hospital_inpatient_bed_occupied_7_day_avg',
 'inpatient_beds_used_covid_7_day_avg',
 'total_adult_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg',
 'total_adult_patients_hospitalized_confirmed_covid_7_day_avg',
 'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_7_day_avg',
 'total_pediatric_patients_hospitalized_confirmed_covid_7_day_avg',
 'inpatient_beds_7_day_avg',
 'total_icu_beds_7_day_avg',
 'total_staffed_adult_icu_beds_7_day_avg',
 'icu_beds_used_7_day_avg',
 'staffed_adult_icu_bed_occupancy_7_day_avg',
 'staffed_icu_adult_patients_confirmed_and_suspected_covid_7_day_avg',
 'staffed_icu_adult_patients_confirmed_covid_7_day_avg',
 'total_p

In [5]:
def ifZero(x):
    if x < 0:
        return 0
    return x

In [6]:
features = ['inpatient_beds_used_covid_7_day_sum', 
            'all_adult_hospital_inpatient_beds_7_day_sum',
            'total_beds_7_day_sum', 
            'inpatient_beds_used_7_day_sum', 
            'inpatient_beds_7_day_sum', 
            'previous_day_admission_adult_covid_confirmed_7_day_sum',
            'previous_day_admission_adult_covid_suspected_7_day_sum']

In [7]:
sliced = data[features]
sliced

,inpatient_beds_used_covid_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,total_beds_7_day_sum,inpatient_beds_used_7_day_sum,inpatient_beds_7_day_sum,previous_day_admission_adult_covid_confirmed_7_day_sum,previous_day_admission_adult_covid_suspected_7_day_sum
0,0.0,NaN,280.0,268.0,280.0,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,436.0,NaN,NaN,NaN,NaN,NaN,NaN
3,71.0,NaN,2676.0,422.0,679.0,NaN,NaN
4,35.0,NaN,1736.0,536.0,1554.0,NaN,NaN
...,...,...,...,...,...,...,...
29995,34.0,861.0,1365.0,321.0,861.0,0.0,15.0
29996,6.0,171.0,75.0,135.0,175.0,-999999.0,0.0
29997,20.0,175.0,210.0,58.0,175.0,5.0,-999999.0
29998,0.0,95.0,168.0,22.0,105.0,0.0,0.0


In [8]:
psliced = sliced.dropna()
psliced= psliced.applymap(lambda x: ifZero(x))
psliced

,inpatient_beds_used_covid_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,total_beds_7_day_sum,inpatient_beds_used_7_day_sum,inpatient_beds_7_day_sum,previous_day_admission_adult_covid_confirmed_7_day_sum,previous_day_admission_adult_covid_suspected_7_day_sum
39,0.0,154.0,154.0,40.0,154.0,0.0,0.0
167,0.0,77.0,112.0,36.0,77.0,0.0,0.0
188,0.0,70.0,100.0,0.0,70.0,0.0,0.0
192,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201,0.0,175.0,266.0,72.0,175.0,0.0,0.0
...,...,...,...,...,...,...,...
29995,34.0,861.0,1365.0,321.0,861.0,0.0,15.0
29996,6.0,171.0,75.0,135.0,175.0,0.0,0.0
29997,20.0,175.0,210.0,58.0,175.0,5.0,0.0
29998,0.0,95.0,168.0,22.0,105.0,0.0,0.0


In [9]:
len(psliced)

15492

In [10]:
input = psliced['previous_day_admission_adult_covid_confirmed_7_day_sum'].values.reshape(-1, 1)
output = psliced['inpatient_beds_used_covid_7_day_sum'].values.reshape(-1, 1)

In [11]:
xtrain, xtest, ytrain, ytest = train_test_split(input, output, test_size=0.3, random_state=404)

In [12]:
dtree = DecisionTreeClassifier(random_state=404)
dtree.fit(xtrain, ytrain)

DecisionTreeClassifier(random_state=404)

In [13]:
y_pred = dtree.predict(xtest)

print("Train data accuracy:",accuracy_score(y_true = ytrain, y_pred = dtree.predict(xtrain)))
print("Test data accuracy:",accuracy_score(y_true = ytest, y_pred = y_pred))

Train data accuracy: 0.3346551088159351
Test data accuracy: 0.30787435456110157
